**Tabla de contenido**

- [Examinando la arquitectura ARLSTM](#Examinando-la-arquitectura-ARLSTM)

En el último capítulo, examinamos y construimos una red neuronal convolucional (CNN). Incluso la combinamos con la arquitectura LSTM para probar si podíamos superar a los modelos LSTM. Los resultados fueron mixtos, ya que los modelos CNN tuvieron un peor rendimiento como modelos de un solo paso, el mejor rendimiento como modelos de múltiples pasos y un rendimiento igualmente bueno como modelos de múltiples salidas.

Ahora nos centraremos completamente en los modelos de múltiples pasos, ya que todos ellos generan toda la secuencia de predicciones de una sola vez. Vamos a modificar ese comportamiento y a generar gradualmente la secuencia de predicciones, utilizando predicciones pasadas para hacer nuevas predicciones. De esa manera, el modelo creará pronósticos en tiempo real, pero utilizando sus propias predicciones para informar la salida.

Esta arquitectura se utiliza comúnmente con LSTM y se llama LSTM autorregresivo (ARLSTM). En este capítulo, primero exploraremos la arquitectura general del modelo ARLSTM y luego lo construiremos en Keras para ver si podemos construir un nuevo modelo de múltiples pasos de alto rendimiento.

# Examinando la arquitectura ARLSTM

Hemos construido muchos modelos de múltiples pasos que todos generan predicciones para el volumen de tráfico en las próximas 24 horas. Cada modelo ha generado toda la secuencia de predicción de una sola vez, lo que significa que obtenemos 24 valores del modelo de inmediato.

A efectos de ilustración, consideremos un modelo simple con solo una capa LSTM. La figura 17.1 muestra la arquitectura general de los modelos de múltiples pasos que hemos construido hasta ahora. Cada uno de ellos tenía entradas que llegaban, pasando a través de una capa, ya sea LSTM, Densa o Conv1D, y resultando en una secuencia de 24 valores. Este tipo de arquitectura obliga a una salida de 24 valores.

![Arquitectura](fig/arMultiStep.png)

La Figura 17.1 ilustra un modelo multistep de disparo único con una capa LSTM. Todos los modelos multistep que hemos construido han tenido esta arquitectura general. La capa LSTM puede ser reemplazada fácilmente por una capa CNN o una capa densa.

¿Pero qué pasa si queremos una secuencia más larga? ¿O una secuencia más corta? ¿Qué pasa si deseamos predecir solo las próximas 8 horas, o predecir las próximas 48 horas? En ese caso, debemos rehacer nuestras ventanas de datos y volver a entrenar los modelos, lo cual podría representar bastante trabajo.

En cambio, podemos optar por un modelo de aprendizaje profundo autorregresivo. Como puede ver en la figura 17.2, cada predicción se envía de vuelta al modelo, lo que le permite generar la siguiente predicción. Este proceso se repite hasta obtener una secuencia de la longitud deseada.

![ARLSTM](fig/ARLSTM.png)

Figura 17.2 *Un modelo LSTM autorregresivo. Este modelo devuelve una primera predicción en t24, y se envía de vuelta al modelo para generar la predicción en t25. Este proceso se repite hasta obtener la longitud de salida deseada. Nuevamente, se muestra una capa LSTM, pero podría ser una CNN o una capa densa.*

Puede ver lo fácil que resulta generar cualquier longitud de secuencia utilizando una arquitectura de aprendizaje profundo autorregresiva. Este enfoque tiene la ventaja añadida de permitirnos pronosticar series temporales con diferentes escalas, como horas, días o meses, evitando al mismo tiempo tener que volver a entrenar a un nuevo modelo. Este es el tipo de arquitectura construida por Google DeepMind para crear WaveNet (https://deepmind.com/blog/article/wavenet-generative-model-raw-audio), un modelo que genera secuencias de audio sin procesar. En el contexto de las series temporales, DeepAR (http://mng.bz/GEoV) es una metodología que también utiliza una red neuronal recurrente autorregresiva para lograr resultados de última generación.

`Sin embargo, los modelos autorregresivos de aprendizaje profundo vienen con una advertencia importante, que es la acumulación de errores`. Hemos pronosticado muchas series temporales y sabemos que siempre hay alguna discrepancia entre nuestras predicciones y los valores reales. Ese error se acumula a medida que se retroalimenta al modelo, lo que significa que las predicciones posteriores tendrán un error mayor que las predicciones anteriores. `Por lo tanto, aunque la arquitectura autorregresiva de aprendizaje profundo parece poderosa, puede que no sea la mejor solución para un problema en particular. De ahí la importancia de utilizar un protocolo de pruebas riguroso, que es realmente lo que hemos desarrollado desde el capítulo 13.`

Aún así, es bueno tener este modelo en su caja de herramientas de métodos de pronóstico de series temporales. En la siguiente sección, codificaremos un modelo LSTM autorregresivo para producir pronósticos para las próximas 24 horas. Compararemos su rendimiento con el de nuestros modelos anteriores de varios pasos.
